# Import Libraries and Setup Logging

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import joblib
from datetime import datetime, timedelta
import os
import logging
from typing import Dict, List, Tuple, Optional, Any, Union

# Set up logging
def setup_logging():
    """Set up logging configuration"""
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.StreamHandler(),
            logging.FileHandler('ticket_predictor.log')
        ]
    )
    return logging.getLogger(__name__)

# Global logger
logger = setup_logging()

# Data Processing Functions

In [3]:
def load_and_prepare_data(data_file: str = 'train_data.csv') -> Tuple[pd.DataFrame, List[str], List[str], List[str]]:
    """
    Load and prepare the dataset with appropriate preprocessing
    
    Args:
        data_file: Path to the CSV dataset
        
    Returns:
        Tuple containing:
        - Processed DataFrame
        - List of categorical columns
        - List of numerical columns
        - List of all columns
    """
    logger.info(f"Loading data from {data_file}")
    try:
        # Load dataset
        data = pd.read_csv(data_file)
        
        # Handle missing values
        handle_missing_values(data)
        
        # Calculate booking lead time if date columns exist
        calculate_lead_time(data)
        
        # Drop unnecessary columns
        drop_columns = ['PNR Number', 'Current Status', 'Date of Journey', 'Booking Date', 'Age of Passengers' ]
        data.drop(columns=[col for col in drop_columns if col in data.columns], inplace=True)
        
        # Convert target variable to binary
        target_column = 'Confirmation Status'
        if target_column in data.columns:
            data[target_column] = data[target_column].map(
                {'Confirmed': 1, 'Not Confirmed': 0, 'Confirmed (RAC)': 1, 'Waitlisted': 0}
            )
        
        # Identify column types
        categorical_cols = data.select_dtypes(include=['object', 'category']).columns.tolist()
        numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
        if target_column in numerical_cols:
            numerical_cols.remove(target_column)
        all_columns = list(data.columns)
        if target_column in all_columns:
            all_columns.remove(target_column)
            
        logger.info(f"Data loaded successfully with {len(data)} records")
        return data, categorical_cols, numerical_cols, all_columns
        
    except Exception as e:
        logger.error(f"Error loading data: {str(e)}")
        raise

In [4]:
def handle_missing_values(data: pd.DataFrame) -> None:
    """
    Handle missing values in the dataset
    
    Args:
        data: DataFrame to process
    """
    # Handle numeric columns
    numeric_cols = data.select_dtypes(include=['int64', 'float64']).columns
    data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].median())
    
    # Handle categorical columns
    categorical_cols = data.select_dtypes(include=['object', 'category']).columns
    for col in categorical_cols:
        data[col] = data[col].fillna(data[col].mode().iloc[0])

In [8]:
def calculate_lead_time(data: pd.DataFrame) -> None:
    """
    Calculate booking lead time from date columns
    
    Args:
        data: DataFrame containing date columns
    """
    if 'Date of Journey' in data.columns and 'Booking Date' in data.columns:
        data['Booking_Lead_Time'] = (
            pd.to_datetime(data['Date of Journey'], errors='coerce') - 
            pd.to_datetime(data['Booking Date'], errors='coerce')
        ).dt.days
        
        # Handle invalid dates resulting in NaN lead times
        data['Booking_Lead_Time'] = data['Booking_Lead_Time'].fillna(data['Booking_Lead_Time'].median())

# Model building functions

In [11]:
def create_preprocessor(categorical_cols: List[str], numerical_cols: List[str]) -> ColumnTransformer:
    """
    Create preprocessing pipeline for features
    
    Args:
        categorical_cols: List of categorical column names
        numerical_cols: List of numerical column names
        
    Returns:
        ColumnTransformer for data preprocessing
    """
    return ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_cols),
            ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)
        ]
    )

In [13]:
def build_model(data: pd.DataFrame, categorical_cols: List[str], numerical_cols: List[str], 
                target_column: str = 'Confirmation Status', model_file: str = 'train_ticket_model.pkl') -> Tuple[Pipeline, pd.DataFrame, pd.Series]:
    """
    Build and train the prediction model using a pipeline
    
    Args:
        data: Prepared DataFrame
        categorical_cols: List of categorical column names
        numerical_cols: List of numerical column names
        target_column: Name of target column
        model_file: Path to save the trained model
        
    Returns:
        Tuple containing:
        - Trained model pipeline
        - Test features
        - Test labels
    """
    logger.info("Building and training model")
    try:
        # Split features and target
        X = data.drop(columns=[target_column])
        y = data[target_column]
        
        # Create preprocessing pipeline
        preprocessor = create_preprocessor(categorical_cols, numerical_cols)
        
        # Create full pipeline with preprocessing and model
        model_pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', RandomForestClassifier(
                n_estimators=100, 
                random_state=42, 
                n_jobs=-1,
                class_weight='balanced'
            ))
        ])
        
        # Split data and train model
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
        
        model_pipeline.fit(X_train, y_train)
        
        # Evaluate model
        evaluate_model(model_pipeline, X_test, y_test)
        
        # Save model for future use
        joblib.dump(model_pipeline, model_file)
        logger.info(f"Model saved as '{model_file}'")
        
        return model_pipeline, X_test, y_test
        
    except Exception as e:
        logger.error(f"Error building model: {str(e)}")
        raise

In [15]:
def evaluate_model(model: Pipeline, X_test: pd.DataFrame, y_test: pd.Series) -> None:
    """
    Evaluate model performance
    
    Args:
        model: Trained model pipeline
        X_test: Test features
        y_test: Test labels
    """
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    logger.info(f"Model trained. Accuracy: {accuracy:.4f}")
    logger.info("\nClassification Report:\n" + classification_report(y_test, y_pred))


In [17]:
def load_or_train_model(data_file: str = 'train_data.csv', model_file: str = 'train_ticket_model1.pkl') -> Tuple[Pipeline, pd.DataFrame, List[str], List[str], List[str]]:
    """
    Load existing model or train new one if not available
    
    Args:
        data_file: Path to the CSV dataset
        model_file: Path to save/load the trained model
        
    Returns:
        Tuple containing:
        - Model pipeline
        - Prepared data
        - Categorical columns
        - Numerical columns
        - All columns
    """
    try:
        # Load and prepare data
        data, categorical_cols, numerical_cols, all_columns = load_and_prepare_data(data_file)
        target_column = 'Confirmation Status'
        
        # Try to load existing model
        if os.path.exists(model_file):
            logger.info("Loading existing model...")
            model_pipeline = joblib.load(model_file)
            logger.info("Existing model loaded successfully")
        else:
            logger.info("No existing model found. Training new model...")
            model_pipeline, _, _ = build_model(
                data, categorical_cols, numerical_cols, target_column, model_file
            )
            
        return model_pipeline, data, categorical_cols, numerical_cols, all_columns
            
    except Exception as e:
        logger.error(f"Error loading or training model: {str(e)}")
        raise

# Prediction functions

In [20]:
def get_prediction(model: Pipeline, user_input: Dict[str, Any]) -> Tuple[Optional[int], Optional[float]]:
    """
    Make prediction based on user input
    
    Args:
        model: Trained model pipeline
        user_input: Dictionary of feature values for prediction
        
    Returns:
        Tuple containing prediction (0/1) and probability
    """
    try:
        # Convert to DataFrame
        user_df = pd.DataFrame([user_input])
        
        # Make prediction (pipeline handles preprocessing)
        prediction = model.predict(user_df)
        probability = model.predict_proba(user_df)[0][1]
        
        # Return prediction results
        return prediction[0], probability
        
    except Exception as e:
        logger.error(f"Error during prediction: {str(e)}")
        return None, None

In [22]:
def display_prediction_result(prediction: Optional[int], probability: Optional[float]) -> None:
    """
    Display prediction result with formatting and interpretation
    
    Args:
        prediction: Binary prediction (0/1) or None
        probability: Confidence probability or None
    """
    if prediction is None or probability is None:
        print("No prediction available.")
        return
    
    display_header("PREDICTION RESULT", char="-")
    
    # Format and display results
    status = 'Confirmed' if prediction == 1 else 'Not Confirmed'
    
    print(f"Predicted Status: {status}")
    print(f"Confidence: {probability:.2%}")
    
    # Display interpretation with helpful advice
    display_interpretation(probability)

In [24]:
def display_interpretation(probability: float) -> None:
    """
    Display interpretation and recommendations based on probability
    
    Args:
        probability: Confidence probability
    """
    print("\nInterpretation:")
    
    if probability > 0.8:
        print("Very likely to be confirmed")
        print("Recommendation: You can be confident about this ticket being confirmed.")
    elif probability > 0.6:
        print("Likely to be confirmed")
        print("Recommendation: Good chance of confirmation, but keep checking status.")
    elif probability > 0.4:
        print("Uncertain confirmation status")
        print("Recommendation: Consider a backup travel plan just in case.")
    elif probability > 0.2:
        print("Likely to remain waitlisted")
        print("Recommendation: Look for alternatives if travel is important.")
    else:
        print("Very likely to remain waitlisted")
        print("Recommendation: Consider booking an alternative confirmed ticket.")

# User input collection functions

In [27]:
def collect_user_input(data: pd.DataFrame, all_columns: List[str], 
                      categorical_cols: List[str], numerical_cols: List[str]) -> Optional[Dict[str, Any]]:
    """
    Collect user input for prediction
    
    Args:
        data: DataFrame with training data
        all_columns: List of all feature columns
        categorical_cols: List of categorical column names
        numerical_cols: List of numerical column names
        
    Returns:
        Dictionary of user inputs or None if cancelled
    """
    display_header("TRAIN TICKET CONFIRMATION PREDICTOR")
    print("Enter details for prediction:")
    
    try:
        # Define fields to skip asking for
        skip_fields = get_skip_fields()
        skip_fields_lower = [field.lower() for field in skip_fields]
        
        # Get date information
        dates_info = collect_date_information()
        if not dates_info:
            return None
            
        booking_date, travel_date, booking_lead_time = dates_info
        
        # Initialize user input dictionary with the calculated lead time
        user_input = {
            'Booking_Lead_Time': booking_lead_time
        }
        
        # Handle Special Considerations separately
        special_considerations = select_special_considerations()
        user_input['Special Considerations'] = special_considerations
        
        # Process each column with appropriate options
        collect_feature_inputs(user_input, all_columns, categorical_cols, numerical_cols, 
                             data, skip_fields_lower)
        
        # Display summary and confirm
        if not display_summary_and_confirm(user_input, booking_date, travel_date, skip_fields_lower):
            return None
        
        return user_input
        
    except Exception as e:
        logger.error(f"Error collecting user input: {str(e)}")
        return None

In [29]:
def get_skip_fields() -> List[str]:
    """Get list of fields to skip in user input collection"""
    return [
        'Train Type', 'Train Number', 'Travel Distance', 'Distance', 
        'Travel Time', 'Seat Availability', 'Available Seats',
        'Number of Passengers', 'Passengers', 'Number of Stations', 'Stations',
        'Waitlist Position', 'Waitlist', 'Special Considerations'
    ]

In [31]:
def collect_date_information() -> Optional[Tuple[str, str, int]]:
    """
    Collect booking and travel date information
    
    Returns:
        Tuple of booking date, travel date, and lead time or None if cancelled
    """
    # Get current date for validation
    today = datetime.now().date()
    today_str = today.strftime('%Y-%m-%d')
    
    # Calculate the maximum allowed travel date (60 days from today)
    max_travel_date = today + timedelta(days=60)
    
    # Get booking date
    booking_date = get_date_input(
        "Enter Booking Date (YYYY-MM-DD)", 
        default=today_str, 
        min_date=today
    )
    
    if booking_date is None:
        return None
        
    booking_date_obj = pd.to_datetime(booking_date)
    
    # Get travel date with maximum limit of 60 days from today
    print("\nNOTE: You can only book tickets for travel dates within the next 60 days")
    travel_date = get_date_input(
        "Enter Travel Date (YYYY-MM-DD)", 
        default=None,
        min_date=booking_date_obj.date(),
        max_date=max_travel_date
    )
    
    if travel_date is None:
        return None
        
    travel_date_obj = pd.to_datetime(travel_date)
    
    # Calculate booking lead time
    booking_lead_time = (travel_date_obj - booking_date_obj).days
    
    # Enforce maximum lead time of 60 days
    if booking_lead_time > 60:
        print("\nWarning: Maximum booking lead time is 60 days. Adjusting to 60 days.")
        booking_lead_time = 60
        # Adjust travel date to match the 60-day limit
        travel_date_obj = booking_date_obj + timedelta(days=60)
        travel_date = travel_date_obj.strftime('%Y-%m-%d')
    
    print(f"\nBooking Lead Time: {booking_lead_time} days")
    
    return booking_date, travel_date, booking_lead_time

In [33]:
def collect_feature_inputs(user_input: Dict[str, Any], all_columns: List[str], 
                          categorical_cols: List[str], numerical_cols: List[str], 
                          data: pd.DataFrame, skip_fields_lower: List[str]) -> None:
    """
    Collect inputs for each feature
    
    Args:
        user_input: Dictionary to store collected inputs
        all_columns: List of all feature columns
        categorical_cols: List of categorical column names
        numerical_cols: List of numerical column names
        data: DataFrame with training data
        skip_fields_lower: List of lowercase field names to skip
    """
    for col in all_columns:
        # Skip fields already processed or in skip list
        if (col in user_input or 
            col == 'Booking_Lead_Time' or
            col == 'Special Considerations' or
            any(skip_term in col.lower() for skip_term in skip_fields_lower)):
            
            # For skipped fields, add a default value
            if col not in user_input:
                if col in numerical_cols:
                    user_input[col] = data[col].mean() if col in data else 0.0
                else:
                    user_input[col] = data[col].mode()[0] if col in data else "Unknown"
            
            continue
        
        # Get options for this column
        options = get_options_list(data, col)
        
        # Handle column with available options
        if options and len(options) > 1:
            user_input[col] = select_from_options(options, f"Select {col}:")
        
        # Handle column without available options
        elif col in categorical_cols:
            user_input[col] = input(f"Enter {col}: ")
        
        # Handle numerical fields
        elif col in numerical_cols:
            user_input[col] = get_numeric_input(col, data)
    
    # Ensure all columns from training data are represented
    for col in all_columns:
        if col not in user_input:
            if col in numerical_cols:
                user_input[col] = 0.0
            else:
                user_input[col] = "Unknown"

In [35]:
def get_numeric_input(column: str, data: pd.DataFrame) -> float:
    """
    Get numeric input from user
    
    Args:
        column: Column name
        data: DataFrame with training data
            
    Returns:
        Numeric value
    """
    while True:
        try:
            value = input(f"Enter {column} (numeric): ")
            if value.strip():
                return float(value)
            else:
                # Use mean value from data if available
                default = data[column].mean() if column in data else 0.0
                print(f"Using default value: {default}")
                return default
        except ValueError:
            print("Please enter a valid number.")

In [37]:
def display_summary_and_confirm(user_input: Dict[str, Any], booking_date: str, 
                               travel_date: str, skip_fields_lower: List[str]) -> bool:
    """
    Display summary of inputs and confirm prediction
    
    Args:
        user_input: Dictionary of collected inputs
        booking_date: Booking date string
        travel_date: Travel date string
        skip_fields_lower: List of lowercase fields to skip
            
    Returns:
        True if user confirms, False otherwise
    """
    display_header("INPUT SUMMARY", char="-")
    
    # Display sorted inputs
    for key, value in sorted(user_input.items()):
        # Skip displaying fields that were excluded from user input
        if not any(skip_term in key.lower() for skip_term in skip_fields_lower) or key == 'Special Considerations':
            print(f"{key}: {value}")
    
    # Add booking and travel dates to summary
    print(f"Booking Date: {booking_date}")
    print(f"Travel Date: {travel_date}")
    
    # Confirm prediction
    confirm = input("\nProceed with prediction? (y/n): ")
    if confirm.lower() != 'y':
        print("Prediction cancelled.")
        return False
        
    return True

In [39]:
def get_options_list(data: pd.DataFrame, column: str) -> List[Any]:
    """
    Extract available options from dataset for a specific column
    
    Args:
        data: DataFrame with training data
        column: Column name
        
    Returns:
        List of unique values for the column
    """
    if column in data.columns:
        options = sorted(data[column].dropna().unique().tolist())
        return options
    return []

In [41]:
def select_from_options(options: List[Any], prompt: str) -> Any:
    """
    Display options and let user select one
    
    Args:
        options: List of available options
        prompt: Prompt text
        
    Returns:
        Selected option
    """
    print(f"\n{prompt}")
    
    # Display options vertically
    display_options_vertical(options)
    
    while True:
        try:
            selection = input("\nEnter number or type directly: ")
            
            # If user entered a number
            if selection.isdigit() and 1 <= int(selection) <= len(options):
                return options[int(selection) - 1]
            
            # If user entered a value directly
            elif selection.upper() in [str(s).upper() for s in options]:
                return next(s for s in options if str(s).upper() == selection.upper())
            
            # If the selection isn't valid but user wants to use it anyway
            else:
                confirm = input(f"'{selection}' is not in the list. Use it anyway? (y/n): ")
                if confirm.lower() == 'y':
                    return selection
                print("Please make a valid selection.")
        except (ValueError, IndexError):
            print("Invalid selection. Please try again.")

In [43]:
def display_options_vertical(options: List[Any]) -> None:
    """
    Display options vertically
    
    Args:
        options: List of options to display
    """
    for i, option in enumerate(options, 1):
        print(f"{i}. {option}")


In [45]:
def select_special_considerations() -> str:
    """
    Display special considerations options and let user select one
    
    Returns:
        Selected special consideration
    """
    options = ["Physically Handicapped", "Senior Citizen", "None"]
    return select_from_options(options, "Select Special Considerations:")

In [47]:
def get_date_input(prompt: str, default: Optional[str] = None, 
                  min_date: Optional[datetime.date] = None, 
                  max_date: Optional[datetime.date] = None) -> Optional[str]:
    """
    Get a date input with validation and default option
    
    Args:
        prompt: Prompt text
        default: Default date value
        min_date: Minimum allowed date
        max_date: Maximum allowed date
        
    Returns:
        Date string in YYYY-MM-DD format or None if cancelled
    """
    # Set minimum date to today if not specified
    if min_date is None:
        min_date = datetime.now().date()
        
    # Format for display
    min_date_str = min_date.strftime('%Y-%m-%d')
    max_date_str = max_date.strftime('%Y-%m-%d') if max_date else "None"
    
    # Display date range if max_date is provided
    date_range_info = f" (valid range: {min_date_str} to {max_date_str})" if max_date else f" (must be on or after {min_date_str})"
    
    while True:
        date_str = input(f"{prompt}{date_range_info}" + (f" (default: {default}): " if default else ": "))
        
        # Allow cancellation
        if date_str.lower() == 'cancel':
            return None
            
        # Use default if provided and input is empty
        if not date_str and default:
            # Validate default date against minimum and maximum dates
            default_date = pd.to_datetime(default, errors='coerce').date()
            if default_date < min_date:
                print(f"Default date {default} is before the minimum allowed date {min_date_str}.")
                print(f"Please enter a date on or after {min_date_str}.")
                continue
            if max_date and default_date > max_date:
                print(f"Default date {default} is after the maximum allowed date {max_date_str}.")
                print(f"Please enter a date on or before {max_date_str}.")
                continue
            return default
            
        try:
            # Validate date format
            date_obj = pd.to_datetime(date_str, errors='raise')
            
            # Validate date is on or after minimum date
            if date_obj.date() < min_date:
                print(f"Error: Date must be on or after {min_date_str}.")
                continue
                
            # Validate date is on or before maximum date if provided
            if max_date and date_obj.date() > max_date:
                print(f"Error: Date must be on or before {max_date_str}.")
                continue
                
            return date_str
        except Exception as e:
            print(f"Invalid date format. Please use YYYY-MM-DD format. Error: {e}")

# Utility functions

In [50]:
def display_header(text: str, char: str = "=", width: int = 60) -> None:
    """
    Display formatted header text
    
    Args:
        text: Header text
        char: Character to use for the border
        width: Width of the header
    """
    print(f"\n{char * width}")
    print(text.center(width))
    print(f"{char * width}")

# Main function

In [ ]:
def main() -> None:
    """Main function to run the application"""


    model_path = "train_ticket_model.pkl"
    if os.path.exists(model_path):
        os.remove(model_path)
        print("Old model deleted.")
    else:
        print("Model file not found, skipping deletion.")

    try:
        # Display app header
        print('=' * 60)
        print('INDIAN RAILWAYS TICKET CONFIRMATION PREDICTOR'.center(60))
        print('=' * 60)
        
        # Load data and model
        model_pipeline, data, categorical_cols, numerical_cols, all_columns = load_or_train_model()
        
        while True:
            # Get prediction for new input
            print("\nReady to make predictions for new data.")
            user_input = collect_user_input(data, all_columns, categorical_cols, numerical_cols)
            
            # Make prediction if input was provided
            if user_input:
                prediction, probability = get_prediction(model_pipeline, user_input)
                # Display results if prediction was made
                if prediction is not None:
                    display_prediction_result(prediction, probability)
            
            # Ask if user wants to make another prediction
            another = input("\nMake another prediction? (y/n): ")
            if another.lower() != 'y':
                print("Thank you for using the Train Ticket Confirmation Predictor!")
                break
        
    except FileNotFoundError as e:
        logger.error(f"File not found: {str(e)}")
        print(f"Error: Dataset file not found. Please ensure the file exists in the current directory.")
    except Exception as e:
        logger.error(f"Unexpected error: {str(e)}", exc_info=True)
        print(f"An unexpected error occurred: {str(e)}")
        print(f"Check the log file for more details.")


if __name__ == "__main__":
    main()

2025-04-03 07:06:08,386 - INFO - Loading data from train_data.csv
2025-04-03 07:06:08,536 - INFO - Data loaded successfully with 10000 records
2025-04-03 07:06:08,538 - INFO - No existing model found. Training new model...
2025-04-03 07:06:08,540 - INFO - Building and training model


Model file not found, skipping deletion.
       INDIAN RAILWAYS TICKET CONFIRMATION PREDICTOR        


2025-04-03 07:06:08,848 - INFO - Model trained. Accuracy: 0.9920
2025-04-03 07:06:08,855 - INFO - 
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       666
           1       0.99      1.00      0.99      1334

    accuracy                           0.99      2000
   macro avg       0.99      0.99      0.99      2000
weighted avg       0.99      0.99      0.99      2000

2025-04-03 07:06:08,888 - INFO - Model saved as 'train_ticket_model1.pkl'



Ready to make predictions for new data.

            TRAIN TICKET CONFIRMATION PREDICTOR             
Enter details for prediction:


Enter Booking Date (YYYY-MM-DD) (must be on or after 2025-04-03) (default: 2025-04-03):  2025-04-05


...


Select Holiday or Peak Season:
1. No
2. Yes



Enter number or type directly:  2



------------------------------------------------------------
                       INPUT SUMMARY                        
------------------------------------------------------------
Booking Channel: Mobile App
Booking_Lead_Time: 57
Class of Travel: 3AC
Destination Station: RJPB
Holiday or Peak Season: Yes
Quota: General
Source Station: BCT
Special Considerations: None
Booking Date: 2025-04-05
Travel Date: 2025-06-01



Proceed with prediction? (y/n):  y



------------------------------------------------------------
                     PREDICTION RESULT                      
------------------------------------------------------------
Predicted Status: Confirmed
Confidence: 100.00%

Interpretation:
Very likely to be confirmed
Recommendation: You can be confident about this ticket being confirmed.
